In [1]:
import dropbox
import os
import numpy as np
import pandas as pd
from scipy.stats import t
import math
import collections
from PIL import Image
from dropbox import DropboxOAuth2FlowNoRedirect
import matplotlib.pyplot as plt
import cv2
import pypyodbc
import sys
import traceback
import time

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, Sequential
from keras.layers import BatchNormalization, concatenate, Input, Dropout, Flatten, Dense
from keras.utils.np_utils import to_categorical  #?
from keras.optimizers import Adam
from keras.metrics import categorical_accuracy

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
cnxn = pypyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                      "Server=028-GUMUS1-SRV;"
                      "Database=KOTON_DB;"
                      "Trusted_Connection=yes;")

In [3]:
access_token='0DQLcsXIojcAAAAAAAgybeB3DOmXKK7bRTLyYwkthbrAiGmpQR4AuGINQkjBXhif'
dbx = dropbox.Dropbox(access_token)

retrieved_image_df = pd.read_excel('C:/Users/Recep/Koton/Data/retrieved_image_list.xlsx', sheet_name='Sheet1')

In [4]:
def data_set(codes, indexes):
    """Takes list of option codes as input and returns related dataset, response set (sales amount)
    and genuine option codes that included in e-commerce table.
    """
    
    if len(codes)>2000:
        numOfBatches = 100
    else:
        numOfBatches = 1
    
    string_tar = "SELECT [OptionCode] \
      ,[Groups] \
  FROM [KOTON_DB].[dbo].[Ratio_table_gruouped] Where ("
    
    codes_split = np.array_split(codes, numOfBatches)
    indexes_split = np.array_split(indexes, numOfBatches)
    
    # to prevent empty targets
    gen_codes = []
    gen_indexes = []
    target = []
    
    for batch in range(numOfBatches):
        _ =[]
        flag2 = True
        for i in codes_split[batch]:
            if flag2:
                _.append("OptionCode='"+i+"'")
                flag2 = False
            else:
                 _.append(" OR OptionCode='"+i+"'")
        query2 = string_tar + ''.join(_) + ')'
        temp = pd.read_sql_query(query2, cnxn, index_col='optioncode')
        target.append(temp)
    target = pd.concat(target)

    for code, index in zip(codes, indexes):
        try:
            target.loc[code]
            gen_codes.append(code)
            gen_indexes.append(index)
        except:
            pass        
        
    
    string_inp = "SELECT * \
  FROM [KOTON_DB].[dbo].[Input_table] \
  WHERE "
    
#     numOfBatches = max(min(len(gen_indexes), numOfBatches), 1)
    numOfBatches = max(min(len(gen_indexes), numOfBatches), 1)
    gen_codes_splited = np.array_split(gen_codes, numOfBatches)
    dataset = []
    
    for batch in range(numOfBatches):
        _ =[]
        flag1 = True
        for i in gen_codes_splited[batch]:
            if flag1:
                _.append("OptionCode='"+i+"'")
                flag1 = False
            else:
                 _.append(" OR OptionCode='"+i+"'")
        query1 = string_inp + ''.join(_)
        dataset.append(pd.read_sql_query(query1, cnxn, index_col='optioncode'))
    
    dataset = pd.concat(dataset)
    
    
    return dataset, target, gen_codes, gen_indexes

In [ ]:
def tarrain(numberSamples, startIndex, batchSize=64, return_data=False):
    errorLog = []
    bn_model = Xception(include_top=False, weights='imagenet', pooling='max')

    first = Input(shape=(2048,))

    model = Dense(256, activation='relu')(first)
    model = Dropout(0.5)(model)
    model = Dense(10, activation='softmax')(model)

    model_ = Model(inputs=[first], outputs=model)

    model_.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    
    codes = []
    idx = []
    for picture_index in range(startIndex,startIndex + numberSamples):
        code = retrieved_image_df.iloc[picture_index].name
        codes.append(code)
        idx.append(picture_index)
    
    dset, tar, gen_codes, gen_indexes = data_set(codes, idx)
#     dset = dset.apply(LabelEncoder().fit_transform)
    dset = pd.get_dummies(dset)
    tar = pd.get_dummies(tar)
    print(dset.shape)
    print(tar.shape)
       
    if batchSize>dset.shape[0]:
        batchSize = dset.shape[0]
        print('Batch size is automatically adjusted.')
    
    dset, dset_test, tar, tar_test = train_test_split(dset, tar, test_size=0.01, random_state=42)
    gen_codes, gen_codes_test, gen_indexes, gen_indexes_test = train_test_split(gen_codes, gen_indexes, 
                                                                                test_size=0.01, random_state=42)
    print(dset.shape)
    
    numOfBatches, lastBatch = divmod(dset.shape[0], batchSize-1)
    dsets = np.array_split(dset, numOfBatches)
    tars = np.array_split(tar, numOfBatches)
    gen_codes = np.array_split(gen_codes, numOfBatches)
    gen_indexes = np.array_split(gen_indexes, numOfBatches)
    
    valid_pic1 =[]
    for code, picture_index in zip(gen_codes_test, gen_indexes_test):
        im_index = retrieved_image_df.iloc[picture_index]['image1'+'_index']        
        if (math.isnan(float(im_index))==False):
            filename = str(code).lower()+'_'+'image1'+'_'+str(int(im_index))+'.jpg'
            path = '/Koton_Image_Files/'+filename
            try:
                md, res = dbx.files_download(path)
                temp_pic = cv2.cvtColor(cv2.resize(cv2.imdecode(np.frombuffer(res.content, dtype=np.uint8), -1), (299,299)), cv2.COLOR_BGR2RGB)
                assert temp_pic.shape == (299, 299, 3)
                valid_pic1.append(temp_pic)
            except:
                errorLog.append(path+' Not found')
    valid_pic1 = bn_model.predict(preprocess_input(np.array(valid_pic1)))
#     valid_bottleneck_features = bn_model.predict(valid_pic1)
    
    
    for epoch in range(60):
        tars2 = []
        preds = []
        print('Epoch: '+str(epoch+1)+'/'+str(60))
        for batch in range(numOfBatches):
            pic1 =[]
            print('Batch: '+str(batch+1)+'/'+str(numOfBatches))
            for code, picture_index in zip(gen_codes[batch], gen_indexes[batch]):
                im_index = retrieved_image_df.iloc[picture_index]['image1'+'_index']        
                if (math.isnan(float(im_index))==False):
                    filename = str(code).lower()+'_'+'image1'+'_'+str(int(im_index))+'.jpg'
                    path = '/Koton_Image_Files/'+filename
                    try:
                        md, res = dbx.files_download(path)
                        temp_pic = cv2.cvtColor(cv2.resize(cv2.imdecode(np.frombuffer(res.content, dtype=np.uint8), -1), (299,299)), cv2.COLOR_BGR2RGB)
                        assert temp_pic.shape == (299, 299, 3)
                        pic1.append(temp_pic)
                    except:
                        errorLog.append(path+' Not found')
                        batch += 1 #ignore defective batch
            pic1 = preprocess_input(np.array(pic1))
            try:
                innp = bn_model.predict(pic1)
                model_.fit(innp, tars[batch], validation_data=(valid_pic1, tar_test), epochs=1)
                p = model_.predict(pic1)
                if p.shape == tars[batch].shape:
                    tars2.append(tars[batch])
                    preds.append(p)
                if epoch%3 == 0 and batch==59:
                    acc = categorical_accuracy(np.concatenate(tars2, axis=0 ), np.concatenate(preds, axis=0 ))
                    timestr = time.strftime("%Y%m%d-%H%M%S")
                    model_.save('model'+timestr+'.h5')
                    print("Train accuracy: " + str(acc))
            except:
                exc_type, exc_value, exc_traceback = sys.exc_info()
                lines = traceback.format_exception(exc_type, exc_value, exc_traceback)
                errorLog.append(''.join('!! ' + line for line in lines))
                
    if return_data:
        return model_, errorLog, dset, dset_test, tar, tar_test, gen_codes, gen_codes_test, gen_indexes, gen_indexes_test
    else:
        return model_, errorLog
                    

In [ ]:
model, errorLog, dset, dset_test, tar, tar_test = tarrain(40000, 1724, 64, return_data=True)

(6144, 193)
(6144, 10)
(6082, 193)
Epoch: 1/60
Batch: 1/96
Train on 64 samples, validate on 62 samples
Epoch 1/1
64/64 [==============================] - 1s 9ms/step - loss: 7.3793 - acc: 0.1250 - val_loss: 11.3793 - val_acc: 0.1452
Batch: 2/96
Train on 64 samples, validate on 62 samples
Epoch 1/1
64/64 [==============================] - 0s 359us/step - loss: 10.8278 - acc: 0.1094 - val_loss: 5.6524 - val_acc: 0.1290
Batch: 3/96
Train on 64 samples, validate on 62 samples
Epoch 1/1
64/64 [==============================] - 0s 359us/step - loss: 6.9434 - acc: 0.2031 - val_loss: 5.8268 - val_acc: 0.2097
Batch: 4/96
Batch: 5/96
Train on 64 samples, validate on 62 samples
Epoch 1/1
64/64 [==============================] - 0s 344us/step - loss: 6.6524 - acc: 0.2188 - val_loss: 5.0938 - val_acc: 0.2097
Batch: 6/96
Train on 64 samples, validate on 62 samples
Epoch 1/1
64/64 [==============================] - 0s 375us/step - loss: 5.0562 - acc: 0.1719 - val_loss: 3.8957 - val_acc: 0.1613
Batch:

64/64 [==============================] - 0s 359us/step - loss: 2.1356 - acc: 0.1875 - val_loss: 2.0977 - val_acc: 0.2419
Batch: 10/96
Train on 64 samples, validate on 62 samples
Epoch 1/1
64/64 [==============================] - 0s 359us/step - loss: 2.1413 - acc: 0.1875 - val_loss: 2.1021 - val_acc: 0.2097
Batch: 11/96
Train on 64 samples, validate on 62 samples
Epoch 1/1
64/64 [==============================] - 0s 359us/step - loss: 2.0614 - acc: 0.2031 - val_loss: 2.1449 - val_acc: 0.1774
Batch: 12/96
Batch: 13/96
Train on 64 samples, validate on 62 samples
Epoch 1/1
64/64 [==============================] - 0s 375us/step - loss: 2.2864 - acc: 0.2656 - val_loss: 2.1222 - val_acc: 0.2419
Batch: 14/96
Train on 64 samples, validate on 62 samples
Epoch 1/1
64/64 [==============================] - 0s 359us/step - loss: 2.2637 - acc: 0.2344 - val_loss: 2.1034 - val_acc: 0.2097
Batch: 15/96
Train on 64 samples, validate on 62 samples
Epoch 1/1
64/64 [==============================] - 0s 374

63/63 [==============================] - 0s 381us/step - loss: 2.3323 - acc: 0.1905 - val_loss: 2.0567 - val_acc: 0.1774
Batch: 65/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 0s 381us/step - loss: 2.0674 - acc: 0.2063 - val_loss: 2.0671 - val_acc: 0.1613
Batch: 66/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 0s 365us/step - loss: 2.2634 - acc: 0.1746 - val_loss: 2.0646 - val_acc: 0.1935
Batch: 67/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 0s 381us/step - loss: 2.0720 - acc: 0.2381 - val_loss: 2.0511 - val_acc: 0.2097
Batch: 68/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 0s 381us/step - loss: 2.2411 - acc: 0.1587 - val_loss: 2.0766 - val_acc: 0.1452
Batch: 69/96
Batch: 70/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 0s 365

63/63 [==============================] - 0s 365us/step - loss: 2.1277 - acc: 0.2381 - val_loss: 2.0725 - val_acc: 0.1774
Batch: 73/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 0s 365us/step - loss: 2.1133 - acc: 0.1905 - val_loss: 2.0856 - val_acc: 0.1774
Batch: 74/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 0s 365us/step - loss: 2.1099 - acc: 0.2063 - val_loss: 2.0530 - val_acc: 0.1774
Batch: 75/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 0s 349us/step - loss: 2.1315 - acc: 0.2063 - val_loss: 2.0813 - val_acc: 0.1935
Batch: 76/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 0s 365us/step - loss: 2.1504 - acc: 0.1746 - val_loss: 2.0667 - val_acc: 0.2258
Batch: 77/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 0s 381us/step - los

64/64 [==============================] - 0s 359us/step - loss: 1.9559 - acc: 0.2812 - val_loss: 2.0722 - val_acc: 0.2258
Batch: 29/96
Train on 64 samples, validate on 62 samples
Epoch 1/1
64/64 [==============================] - 0s 359us/step - loss: 2.0169 - acc: 0.2656 - val_loss: 2.0734 - val_acc: 0.2419
Batch: 30/96
Train on 64 samples, validate on 62 samples
Epoch 1/1
64/64 [==============================] - 0s 359us/step - loss: 2.0476 - acc: 0.1719 - val_loss: 2.0814 - val_acc: 0.2097
Batch: 31/96
Batch: 32/96
Train on 64 samples, validate on 62 samples
Epoch 1/1
64/64 [==============================] - 0s 359us/step - loss: 1.9479 - acc: 0.1875 - val_loss: 2.1520 - val_acc: 0.1774
Batch: 33/96
Train on 64 samples, validate on 62 samples
Epoch 1/1
64/64 [==============================] - 0s 391us/step - loss: 2.0364 - acc: 0.3281 - val_loss: 2.1249 - val_acc: 0.1935
Batch: 34/96
Train on 64 samples, validate on 62 samples
Epoch 1/1
64/64 [==============================] - 0s 359

63/63 [==============================] - 0s 349us/step - loss: 2.0733 - acc: 0.2063 - val_loss: 2.1056 - val_acc: 0.1935
Batch: 37/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 0s 397us/step - loss: 2.2114 - acc: 0.1905 - val_loss: 2.0978 - val_acc: 0.1935
Batch: 38/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 0s 365us/step - loss: 2.1446 - acc: 0.1587 - val_loss: 2.0964 - val_acc: 0.1613
Batch: 39/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 0s 381us/step - loss: 2.0871 - acc: 0.1905 - val_loss: 2.1043 - val_acc: 0.1774
Batch: 40/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 0s 381us/step - loss: 2.0918 - acc: 0.2381 - val_loss: 2.0951 - val_acc: 0.1935
Batch: 41/96
Batch: 42/96
Batch: 43/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [===========================

63/63 [==============================] - 0s 365us/step - loss: 2.0347 - acc: 0.2381 - val_loss: 2.0913 - val_acc: 0.2258
Batch: 47/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 0s 365us/step - loss: 2.0267 - acc: 0.2857 - val_loss: 2.0598 - val_acc: 0.2258
Batch: 48/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 0s 365us/step - loss: 1.9821 - acc: 0.3175 - val_loss: 2.0601 - val_acc: 0.2419
Batch: 49/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 0s 349us/step - loss: 2.0258 - acc: 0.2698 - val_loss: 2.0644 - val_acc: 0.2581
Batch: 50/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 0s 365us/step - loss: 1.9735 - acc: 0.2540 - val_loss: 2.0802 - val_acc: 0.2581
Batch: 51/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 0s 365us/step - los

Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 0s 365us/step - loss: 2.1974 - acc: 0.1429 - val_loss: 2.0872 - val_acc: 0.2097
Batch: 56/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 0s 381us/step - loss: 1.9915 - acc: 0.2857 - val_loss: 2.0748 - val_acc: 0.2419
Batch: 57/96
Batch: 58/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 0s 381us/step - loss: 1.9610 - acc: 0.2063 - val_loss: 2.1091 - val_acc: 0.1935
Batch: 59/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 0s 369us/step - loss: 1.9771 - acc: 0.3492 - val_loss: 2.1482 - val_acc: 0.2097
Batch: 60/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 0s 365us/step - loss: 1.9398 - acc: 0.3016 - val_loss: 2.1393 - val_acc: 0.2258
Batch: 61/96
Train on 63 samples, validate on 62 samples
Epo

64/64 [==============================] - 0s 375us/step - loss: 2.0261 - acc: 0.2656 - val_loss: 2.0386 - val_acc: 0.2581
Batch: 8/96
Batch: 9/96
Train on 64 samples, validate on 62 samples
Epoch 1/1
64/64 [==============================] - 0s 375us/step - loss: 2.0844 - acc: 0.2344 - val_loss: 2.0298 - val_acc: 0.2419
Batch: 10/96
Train on 64 samples, validate on 62 samples
Epoch 1/1
64/64 [==============================] - 0s 359us/step - loss: 2.0417 - acc: 0.1562 - val_loss: 2.0462 - val_acc: 0.2419
Batch: 11/96
Train on 64 samples, validate on 62 samples
Epoch 1/1
64/64 [==============================] - 0s 344us/step - loss: 1.9302 - acc: 0.2969 - val_loss: 2.0884 - val_acc: 0.1935
Batch: 12/96
Batch: 13/96
Train on 64 samples, validate on 62 samples
Epoch 1/1
64/64 [==============================] - 0s 469us/step - loss: 2.1134 - acc: 0.2188 - val_loss: 2.0637 - val_acc: 0.2419
Batch: 14/96
Train on 64 samples, validate on 62 samples
Epoch 1/1
64/64 [=============================

Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 0s 365us/step - loss: 2.0189 - acc: 0.2063 - val_loss: 2.0924 - val_acc: 0.2419
Batch: 64/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 0s 381us/step - loss: 2.2419 - acc: 0.1905 - val_loss: 2.1137 - val_acc: 0.2097
Batch: 65/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 0s 365us/step - loss: 2.0135 - acc: 0.3016 - val_loss: 2.1046 - val_acc: 0.2097
Batch: 66/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 0s 365us/step - loss: 2.1802 - acc: 0.2540 - val_loss: 2.0713 - val_acc: 0.2097
Batch: 67/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 [==============================] - 0s 381us/step - loss: 2.0004 - acc: 0.2857 - val_loss: 2.0713 - val_acc: 0.2097
Batch: 68/96
Train on 63 samples, validate on 62 samples
Epoch 1/1
63/63 